# AIR-Bench Integration with Risk Atlas Nexus

This notebook demonstrates how to use the AIR-Bench taxonomy integration with Risk Atlas Nexus. AIR-Bench is a regulation-aligned safety benchmark for responsible AI development, featuring a four-tiered taxonomy with 314 risk categories derived from analyzing 8 government regulations and 16 company policies worldwide.

In [3]:
# Install Risk Atlas Nexus if not already installed
# !pip install risk-atlas-nexus

In [8]:
# Import required libraries
from risk_atlas_nexus.library import RiskAtlasNexus
#from risk_atlas_nexus.blocks.inference import create_inference_engine
from risk_atlas_nexus.metadata_base import MappingMethod
import pandas as pd

In [9]:
# Initialize the Risk Atlas Nexus library
ran = RiskAtlasNexus()

[2025-05-02 15:00:25:748] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


## 1. Exploring the AIR-Bench Taxonomy

First, let's explore the AIR-Bench taxonomy structure and see what risks are available.

In [11]:
# Get all taxonomies
taxonomies = ran.get_all_taxonomies()

# Find and display the AIR-Bench taxonomy
airbench_taxonomy = None
for taxonomy in taxonomies:
    if taxonomy.id == "ai-risk-taxonomy":
        airbench_taxonomy = taxonomy
        break

print(f"Taxonomy: {airbench_taxonomy.name}")
print(f"Description: {airbench_taxonomy.description}")
print(f"URL: {airbench_taxonomy.url}")

Taxonomy: The AI Risk Taxonomy (AIR 2024)
Description: An AI risk taxonomy derived from eight government policies from the European Union, United States, and China and 16 company policies worldwide. It identifies 314 unique risk categories organized into a four-tiered taxonomy. This taxonomy encompasses System & Operational Risks, Content Safety Risks, Societal Risks, and Legal & Rights Risks. The taxonomy establishes connections between various descriptions and approaches to risk, highlighting the overlaps and discrepancies between public and private sector conceptions of risk. 
URL: https://arxiv.org/pdf/2406.17864


In [13]:
# Get all AIR-Bench risks
airbench_risks = ran.get_all_risks(taxonomy="ai-risk-taxonomy")
print(f"Number of AIR-Bench risks: {len(airbench_risks)}")

# Display a few sample risks
sample_risks = airbench_risks[:5]
for risk in sample_risks:
    print(f"\nRisk ID: {risk.id}")
    print(f"Name: {risk.name}")
    print(f"Description: {risk.description}")
    print(f"Part of: {risk.isPartOf}")

Number of AIR-Bench risks: 314

Risk ID: ai-risk-taxonomy-network-intrusion
Name: Network intrusion
Description: Network intrusion is defined as whether the model response promotes or assists in unauthorized network intrusion, confidentiality breaches, or other security risks.
Part of: ai-risk-taxonomy-confidentiality

Risk ID: ai-risk-taxonomy-vulnerability-probing
Name: Vulnerability probing
Description: Vulnerability probing is defined as whether the model's response potentially aids in probing or exploiting vulnerabilities, thus posing a security risk.
Part of: ai-risk-taxonomy-confidentiality

Risk ID: ai-risk-taxonomy-spoofing
Name: Spoofing
Description: Spoofing is defined as whether the model response promotes or assists in activities related to security risks, confidentiality breaches, or spoofing.
Part of: ai-risk-taxonomy-confidentiality

Risk ID: ai-risk-taxonomy-spear-phishing
Name: Spear phishing
Description: Spear phishing is defined as whether the model response potenti

## 2. Exploring the Four-Tier Structure

AIR-Bench uses a four-tiered taxonomy structure. Let's explore this hierarchy.

In [ ]:
# Function to explore the AIR-Bench tier structure
def explore_tier_structure():
    # Get the explorer from RAN
    explorer = ran._risk_explorer
    
    # Tier 1 (Top Level)
    tier1_groups = [
        explorer.get_risk_group_by_id("airbench-system-operational-risks"),
        explorer.get_risk_group_by_id("airbench-content-safety-risks"),
        explorer.get_risk_group_by_id("airbench-societal-risks"),
        explorer.get_risk_group_by_id("airbench-legal-rights-risks")
    ]
    
    # Display tier 1 groups and their tier 2 children
    for tier1 in tier1_groups:
        print(f"\n--- Tier 1: {tier1.name} ---")
        
        if hasattr(tier1, "narrowMatch") and tier1.narrowMatch:
            # Get tier 2 groups
            for tier2_id in tier1.narrowMatch[:3]:  # Show only first 3 for brevity
                tier2 = explorer.get_risk_group_by_id(tier2_id)
                print(f"  Tier 2: {tier2.name}")
                
                if hasattr(tier2, "narrowMatch") and tier2.narrowMatch:
                    # Get tier 3 groups
                    for tier3_id in tier2.narrowMatch[:2]:  # Show only first 2 for brevity
                        tier3 = explorer.get_risk_group_by_id(tier3_id)
                        print(f"    Tier 3: {tier3.name}")
                        
                        if hasattr(tier3, "narrowMatch") and tier3.narrowMatch:
                            # Get tier 4 risks
                            for risk_id in tier3.narrowMatch[:2]:  # Show only first 2 for brevity
                                risk = ran.get_risk(id=risk_id)
                                print(f"      Tier 4 (Risk): {risk.name}")
        print("...more...")

# Explore the tier structure
explore_tier_structure()

## 3. Finding Related Risks Across Taxonomies

One of the key benefits of integrating AIR-Bench with Risk Atlas Nexus is the ability to navigate between different risk taxonomies. Let's see how we can find related risks across different taxonomies.

In [ ]:
# Get a specific AIR-Bench risk
risk = ran.get_risk(id="airbench-factual-error")
print(f"Looking for risks related to: {risk.name}")
print(f"Description: {risk.description}")

# Find related risks in other taxonomies
related_risks = ran.get_related_risks(risk=risk)

# Display related risks
print("\nRelated risks:")
if related_risks:
    for related_risk in related_risks:
        print(f"- {related_risk.name} ({related_risk.isDefinedByTaxonomy})")
        print(f"  Description: {related_risk.description[:100]}...")
        print(f"  Relationship type: {get_relationship_type(risk, related_risk)}")
else:
    print("No related risks found. You may need to run the mapping process first.")

def get_relationship_type(risk1, risk2):
    """Helper function to determine relationship type between risks"""
    if hasattr(risk1, "exactMatch") and risk1.exactMatch and risk2.id in risk1.exactMatch:
        return "Exact match"
    elif hasattr(risk1, "closeMatch") and risk1.closeMatch and risk2.id in risk1.closeMatch:
        return "Close match"
    elif hasattr(risk1, "broadMatch") and risk1.broadMatch and risk2.id in risk1.broadMatch:
        return "Broad match"
    elif hasattr(risk1, "narrowMatch") and risk1.narrowMatch and risk2.id in risk1.narrowMatch:
        return "Narrow match"
    elif hasattr(risk1, "relatedMatch") and risk1.relatedMatch and risk2.id in risk1.relatedMatch:
        return "Related match"
    else:
        return "Unknown relationship"

## 4. Generating New Mappings Between AIR-Bench and IBM Risk Atlas

We can generate new mappings between AIR-Bench and other taxonomies like IBM Risk Atlas using semantic similarity or LLM-based inference.

In [ ]:
# Get sample risks from AIR-Bench and IBM Risk Atlas
airbench_sample_risks = airbench_risks[:5]
ibm_risks = ran.get_all_risks(taxonomy="ibm-ai-risk-atlas")
ibm_sample_risks = ibm_risks[:10]

# Create an inference engine (optional - only needed for LLM-based mapping)
# inference_engine = create_inference_engine()

# Generate mappings using semantic similarity
mappings = ran.generate_proposed_mappings(
    new_risks=airbench_sample_risks,
    existing_risks=ibm_sample_risks,
    inference_engine=None,  # Set to inference_engine for LLM-based mapping
    new_prefix="airbench",
    mapping_method=MappingMethod.SEMANTIC
)

# Display generated mappings
print(f"Generated {len(mappings)} mappings\n")
mapping_data = []
for mapping in mappings:
    mapping_data.append({
        "Subject": mapping.subject_label,
        "Relationship": mapping.predicate_id.replace("skos:", ""),
        "Object": mapping.object_label,
        "Confidence": mapping.similarity_score if hasattr(mapping, "similarity_score") else "N/A",
        "Justification": mapping.mapping_justification
    })

# Display as a table
pd.DataFrame(mapping_data)

## 5. Risk Identification with AIR-Bench Taxonomy

We can use Risk Atlas Nexus to identify AIR-Bench risks in AI use cases using LLM-based inference.

In [ ]:
# Create an inference engine for risk identification
# Note: This requires setting up appropriate credentials for your chosen LLM provider
inference_engine = create_inference_engine()

# Define a sample AI use case
usecase = """Our company is building a generative AI model that will write medical content for healthcare websites. 
The model will be trained on a variety of medical textbooks and journals, and will be used to generate articles 
about medical conditions, treatments, and procedures for public health information websites."""

# Identify risks using AIR-Bench taxonomy
identified_risks = ran.identify_risks_from_usecases(
    usecases=[usecase],
    inference_engine=inference_engine,
    taxonomy="airbench"
)

# Display identified risks
print("Identified AIR-Bench risks:")
for risks_list in identified_risks:
    for risk in risks_list:
        print(f"\n- {risk.name}")
        print(f"  Description: {risk.description[:100]}...")
        print(f"  Taxonomy: {risk.isDefinedByTaxonomy}")
        print(f"  Category: {risk.isPartOf}")

## Conclusion

This notebook demonstrated how to use the AIR-Bench taxonomy integration with Risk Atlas Nexus. We explored the taxonomy structure, navigated relationships between different risk taxonomies, generated mappings, and identified risks in AI use cases.

AIR-Bench's regulation-aligned taxonomy provides valuable insights for ensuring AI safety and compliance with emerging government regulations and company policies.